In [1]:
import torch
import random

from util import TokenizerUtil

tokenizer = TokenizerUtil()

input_ids, attention_mask = tokenizer.encode('how are you', max_length=4)

input_ids, attention_mask, tokenizer.decode(input_ids)

(tensor([   2, 1139,  708,    1]), tensor([1, 1, 1, 1]), '<bos>how are<eos>')

In [2]:
from datasets import load_dataset
from transformers import default_data_collator

dataset = load_dataset('json', data_files='dataset/train.json', split='train')

#4,2,4切分,取第0部分
dataset = dataset.select(range(30000))


def f(data):
    #随机生成两种回答
    if random.random() > 0.5:
        data['chosen'] = data['chosen'].swapcase()
    data = data['prompt'] + data['chosen']

    input_ids, attention_mask = tokenizer.encode(data)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': input_ids.clone()
    }


dataset = dataset.map(f, remove_columns=dataset.column_names)

loader = torch.utils.data.DataLoader(dataset,
                                     collate_fn=default_data_collator,
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True)

len(loader), next(iter(loader)).keys()

(7500, dict_keys(['input_ids', 'attention_mask', 'labels']))

In [3]:
#%run 1.model.ipynb
%run 1.model_gemma2.ipynb
from accelerate import Accelerator

model_actor = LlamaForCausalLM()

model_actor.train()

optimizer = torch.optim.Adam(model_actor.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                       T_max=100,
                                                       eta_min=1e-5)

accelerator = Accelerator(mixed_precision='fp16')

loader, model_actor, optimizer, scheduler = accelerator.prepare(
    loader, model_actor, optimizer, scheduler)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [4]:
for epoch in range(5):
    for i, data in enumerate(loader):
        loss, logits = model_actor(**data)
        accelerator.backward(loss)
        accelerator.clip_grad_norm_(model_actor.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

        if i % 100 == 0:
            lr = optimizer.param_groups[-1]['lr']
            print(epoch, i, len(loader), loss.item(), lr)
            logits = logits[0].argmax(1)
            print(tokenizer.decode(logits))

torch.save(model_actor.to('cpu'), 'model/actor')

0 0 7500 12.59760570526123 9.997779521645793e-05
Normalulador헙coversvies長崎 nest NEWS visibly Minim visibly diplomat wildcard葡萄酒上映SQ而出 اصفهانSQ而出 towSQ Κυodosvies dr flexible ECC(^^ Ine Democratic اصفهان而出 Scarlett カーディガン Ine Democratic tow وعلى indoctr헙 GalloKne Segunda nest visiblyGreens visiblyCorrosion wildcardNER십borns souvenirюреcticideJAXBISOLDegree jugacouldnюре oblivionISOLнциGrease更多 Commentaires Commentaires Commentaires Commentaires更多更多 Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires directionsGreaseGreaseGreaseGrease directions Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires Commentaires directions directions更多 Commentaires Commentaires Commentaires Commentaires更多更多更多更多 Commentaires Commentaires CommentairesGrease更多更多更多更多更多更多更多更多 взаимоGreaseGrease
0 100 7500 4.898504257202148 1e-05
Human: context= CREATE TABLE

0 2600 7500 1.416762351989746 0.00010000000000006346
Human: context= CREATE TABLE table_name818180__1 (date VARCHARin_ VARCHAR VARCHAR, date VARCHAR) question= What is the game score when the298?? Assistant:SELECT count from2_ from from TABLE_2600088__1 where DATE = "198<eos>
0 2700 7500 1.4182230234146118 1e-05
Human: context= CREATE TABLE table_name954606__1 (date_team_ VARCHAR, date_team VARCHAR) question= What is the total for of for the  of of ?? Assistant:SELECT COUNT_1_ FROM table_1334111__1 WHERE date_name = "n"""<eos>
0 2800 7500 2.2744314670562744 0.00010000000000002275
Human: context= CREATE TABLE table_name_94 (date VARCHAR VARCHARteam VARCHAR VARCHAR INTEGER_, date VARCHARteam VARCHAR VARCHAR VARCHAR) question= What team has? has of a ?? of ,?????? Assistant:select count from fromTEAM from from from from TABLE_NAME_52 where NAME2 =NAME = = = "s"""""""<eos>
0 2900 7500 1.2085332870483398 1e-05
Human: context= CREATE TABLE table_name_79 (date VARCHAR, date VARCHAR) question=

0 5400 7500 1.3213348388671875 0.00010000000000008342
Human: context= CREATE TABLE table_name701813__1 (date_team_team VARCHAR, incumbent VARCHAR VARCHAR) question= What is the episode numberrence, the episode??? ? Assistant:select count fromTEAM_ from from TABLE_10121266_1 where NAME = = "s"""D<eos>
0 5500 7500 1.5413939952850342 1e-05
Human: context= CREATE TABLE table_name (name_, CREATE TABLE table_name VARCHAR, name_id VARCHAR) question= Find the names of the attended have a.  episode?? the? the ? Assistant:SELECT T2.name FROM table_id_ T1 JOIN__ T2 ON T1.id_ T2.id_id WHERE BY T2.id<eos>
0 5600 7500 0.9608563780784607 0.00010000000000004415
Human: context= CREATE TABLE table_name_9 (date VARCHAR, time VARCHAR, question= What is the lowest number of the for the game is 20 Assistant:select SUM(year) FROM table_name_8 WHERE time = 8<eos>
0 5700 7500 1.2053589820861816 1e-05
Human: context= CREATE TABLE table_name332037_1 (date VARCHAR, date VARCHAR) question= What is the number for t

1 600 7500 0.8733988404273987 1.002220478354211e-05
Human: context= CREATE TABLE table_name_40date VARCHAR, player_par VARCHAR) player_) question= What player has a total par of of than 13 a total of of 77?? Assistant:select PLAYER from TABLE_NAME_5 where TO_PAR = 7 and TOTAL = "298<eos>
1 700 7500 1.3711141347885132 9.997779521648165e-05
Human: context= CREATE TABLE table_name_7 ( (score INTEGERnumber_ VARCHAR, total__, rank VARCHAR VARCHAR_ VARCHAR) rank VARCHAR VARCHAR_ VARCHAR) question= What the lowest number for number a for, 1,  for of 1, a  than 1,:select MIN(total)km_ FROM table_name_68 WHERE rank >km_ = 1 AND rank > <_ < 1<eos>
1 800 7500 0.6565142869949341 1.002220478354213e-05
Human: context= CREATE TABLE table_name_29 (date VARCHAR, points VARCHAR) question= What points have a Opponent of ?tsburgh?? Assistant:select max(POINTS) from TABLE_NAME_86 where OPPONENT = """BURGH"<eos>
1 900 7500 1.1787868738174438 9.997779521644137e-05
Human: context= CREATE TABLE table_name71215

1 3500 7500 1.0320607423782349 9.980028840710218e-05
Human: context= CREATE TABLE table_name_44 (score VARCHAR, year VARCHAR, question= What thevin is a title is is? in?? what is the title? Assistant:select TITLE from TABLE_NAME_99 where TITLE = "UNITEDIFORNIA AND title LOW""" ="""<eos>
1 3600 7500 0.863860011100769 1.019971159286141e-05
Human: context= CREATE TABLE table_name228808__1 (no_, total VARCHARof VARCHAR) question= What many total are the number what the  of Assistant?? what the?? AssistantAssistant:SELECT SUM(total) FROM table_20325360_2 WHERE country =of = "john" AND<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
1 3700 7500 0.7689096331596375 9.980028840722173e-05
Human: context= CREATE TABLE table_name3407011_1 (date VARCHAR, player VARCHAR) question= What playe

...

3 2800 7500 0.967035174369812 1.1786784144543553e-05
Human: context= CREATE TABLE table_name6286605_1 ( (team VARCHAR, winner_classification VARCHAR) general_classification VARCHAR) question= Who is the winner where a Kitebel as the Classification and points?z as the Classification? Assistant:SELECT WINNER from TABLE_23157997_13 where POINTSUNTAINS_CLASSIFICATION = "bHEOTAKLINE" and POINTS_CLASSIFICATION = "bHEOMAS sTLEIO"<eos>
3 2900 7500 1.1410655975341797 9.821321585584176e-05
Human: context= CREATE TABLE table_name237977_1 (original_, successor VARCHAR) question= Name was the value for the district Capitals2rd liga? Assistant:select VACOR from TABLE_2159571_2 where DISTRICT = "3  3RD"<eos>
3 3000 7500 0.43239113688468933 1.1786784144546244e-05
Human: context= CREATE TABLE table_name_31 (date INTEGERsite VARCHAR, date VARCHAR) question= What was the game on October156,,11201? Assistant:select game_site FROM table_name_26 WHERE date = "1990-11818"<eos>
3 3100 7500 0.8094711899757385 

3 5600 7500 0.8232762217521667 1.2202456766735862e-05
Human: context= CREATE TABLE table_name_96 (date VARCHAR, venue VARCHAR) question= What city has a Date of  1? 1998? Assistant:SELECT city FROM table_name_39 WHERE date = "november 9, 1976"<eos>
3 5700 7500 0.6372660398483276 9.779754323344273e-05
Human: context= CREATE TABLE table_name_9 ( (score INTEGER, points VARCHARfor VARCHAR) question= What is the highest number for the for 1??? Assistant:select max(DRAWN) from TABLE_NAME_81 where POINTS_FOR > 545<eos>
3 5800 7500 1.2583757638931274 1.2202456766723304e-05
Human: context= CREATE TABLE table_name_37 (year INTEGER, attendance VARCHAR) question= What Week has an of 5,,555? Assistant:SELECT AVG(week) FROM table_name_97 WHERE attendance = 71,740"<eos>
3 5900 7500 0.5801289081573486 9.733963460321798e-05
Human: context= CREATE TABLE table_name_51 (year_club_team VARCHAR, player VARCHAR) question= What school didclub team has the-askill play for? Assistant:SELECT SCHOOL_CLUB_TEAM fro

4 700 7500 0.5555070042610168 1.3160058135042347e-05
Human: context= CREATE TABLE table_name_11 (date VARCHAR, club VARCHAR, club VARCHAR) question= What club has a Round10 round of of 100 and and6–00 Assistant:select club FROM table_name_39 WHERE round = "2nd round  AND score = "1:0, 3,0"<eos>
4 800 7500 0.4799358546733856 9.629895815560602e-05
Human: context= CREATE TABLE table_name_8 ( (year VARCHAR, player_) question= What is the nationality of the player with?1? Assistant:SELECT NATIONALITY from TABLE_NAME_13 where RANK = = "3"<eos>
4 900 7500 0.3441234529018402 1.370104184421418e-05
Human: context= CREATE TABLE table_name_61 (date VARCHAR, date VARCHAR) question= What was the score on 27 October 2009? Assistant:SELECT SCORE from TABLE_NAME_88 where DATE = "22 OCTOBER 2008"<eos>
4 1000 7500 0.7914469838142395 9.629895815634647e-05
Human: context= CREATE TABLE table_name_7 ( (date VARCHAR, year VARCHARm__ VARCHAR) question= What is a total than 1..8,, ? Assistant:SELECT COUNT FROM 

4 3300 7500 0.40825459361076355 1.3701041844248893e-05
Human: context= CREATE TABLE table_name_11date VARCHAR, attendance VARCHAR, score VARCHAR) question= What Visitor has a visitor of thecouver, and a Score of 1– 1? Assistant:SELECT DECISION from TABLE_NAME_4 where VISITOR = "WINNERCOUVER" and SCORE = "5" 4"<eos>
4 3400 7500 0.5003371834754944 9.62989581552778e-05
Human: context= CREATE TABLE table_name_92 (year VARCHAR, player VARCHAR) question= What many caps did Mind ailes have? Assistant:select SUM(CAPS) FROM table_name_96 WHERE player = "tom leeiles"<eos>
4 3500 7500 0.43410947918891907 1.3701041844200991e-05
Human: context= CREATE TABLE table_name_7 ( (year VARCHAR, score VARCHAR) question= What country is the ke musterørn from? Assistant:select COUNTRY from TABLE_NAME_96 where PLAYER = "TOM EN"Y"<eos>
4 3600 7500 0.4962652921676636 9.629895815663595e-05
Human: context= CREATE TABLE table_name_23 (score INTEGER, points VARCHAR) question= Whatunt the Venue that has a Venue of os

4 6200 7500 0.5813825726509094 9.571721736087811e-05
Human: context= CREATE TABLE table_name_15 (date_team VARCHAR, home VARCHAR) question= What is the away team score the away was at Oval? Assistant:SELECT away_team FROM table_name_30 WHERE venue = "mcg oval"<eos>
4 6300 7500 0.6795721054077148 1.4282782639026364e-05
Human: context= CREATE TABLE table_name_67 (opponent INTEGER, nationality VARCHAR, wins VARCHAR) question= What is the rank from the withs rank of  than 11 wins? Assistant:SELECT rank(rank) FROM table_name_63 WHERE country > 16 AND country = "spa"<eos>
4 6400 7500 0.6362006068229675 9.571721736099264e-05
Human: context= CREATE TABLE table_name3749940_1 (team_pointsbounds VARCHAR, high_assists VARCHAR) question= Who had the high rebounds when the high where which the High were7) did the high assists? Assistant:SELECT high_rebounds FROM table_19778010_5 WHERE high_assists = "B ( (3)"<eos>
4 6500 7500 0.3194226920604706 1.4282782639032517e-05
Human: context= CREATE TABLE tab